In [1]:
import numpy as np
import random

In [2]:
with open("RegulonDB_TF.txt", "r") as R:
    TFB_raw = R.read()
    
with open("RegulonDB_Promoter.txt", "r") as R:
    Promoters_raw = R.read()

In [3]:
TFB = TFB_raw.split("\n")
Promoters = Promoters_raw.split("\n")

Stripped_TFB = []
for x in TFB:
    if len(x) > 0:
        if x[0] != "#":
            Stripped_TFB.append(x.split("\t"))
        
Stripped_Promoters = []
for x in Promoters:
    if len(x) > 0:
        if x[0] != "#":
            Stripped_Promoters.append(x.split("\t"))



In [4]:
CRP_TFB = []
unrelated_TFB = []
for x in Stripped_TFB:
    if x[1] == "CRP":
        CRP_TFB.append(x)
    else:
        unrelated_TFB.append(x)
CRP_Promoters = []
unrelated_Promoters = []

for x in CRP_TFB: # Ow, O(n^2). But, it doesn't seem to matter too much
    for y in Stripped_Promoters:
        if x[9] == y[1]:
            CRP_Promoters.append(y)
        else:
            unrelated_Promoters.append(y)


In [5]:
from tensorflow.keras.layers import Input, Dense, LSTM, Concatenate, Permute, Dropout,  Flatten,  Lambda, Conv1D, AveragePooling1D, Permute
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow import keras as K
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
inputs = Input(shape=(60, 4))
input_loc = Input(shape = (1,))
input_before = Input(shape=(10, 4))
input_crp = Input(shape=(23, 4))
input_after = Input(shape = (10, 4))
input_rules = Input(shape=(12,))

Conv0 = Conv1D(16, kernel_size = 4)(inputs)
F0 = Flatten()(Conv0)

Conv1 = Conv1D(16,kernel_size = 4, kernel_regularizer = "l2")(input_crp)
F1 = Flatten()(Conv1)

combined_inputs = Concatenate(axis=1)([input_before, input_crp, input_after])
Conv3 = Conv1D(16,kernel_size = 4, kernel_regularizer = "l2")(combined_inputs)
F3 = Flatten()(Conv3)

Concat = Concatenate(axis = 1) ([F0, F1, F3, input_rules])
D = Dense(256, activation = "relu")(Concat)
Output = Dense(1, activation = "sigmoid")(D) 

opt =  Adam(lr = 1e-4, amsgrad=True)
model = Model(inputs = [inputs,input_loc, input_before, input_crp, input_after, input_rules], outputs = Output)
model.compile(optimizer=opt,  loss="binary_crossentropy", metrics = ['acc'])


In [7]:
def dna_to_vectors(dna):
    vecs = []
    bases = ["A", "T", "C", "G"] #, 
    for i in range(len(dna)):
        base = dna[i]
        vec = [0 for x in range(4)]
        for ib, _ in enumerate(bases):
            if base == bases[ib]:
                vec[ib] = 1
                break
        vecs.append(vec)
    return vecs


In [8]:
def get_split_dna(dna):
    before = []
    crp = []
    after = []
    phase = 0
    for i in range(len(dna)):
        if phase == 0:
            if dna[i].lower() == dna[i]:
                before.append(dna[i])
            else:
                crp.append(dna[i])
                phase = 1
        elif phase == 1:
            if dna[i].upper() == dna[i]:
                crp.append(dna[i])
            else:
                after.append(dna[i])
                phase = 2
        elif phase == 2:
            after.append(dna[i])
    return "".join(before), "".join(crp), "".join(after)


In [9]:
def split_by_upper(z): 
    for i in range(len(z)):
        if z[i] == z[i].upper():
            return z[:i]
def unison_shuffled_copies(a, b, c, d, e, f, g ):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p], d[p], e[p], f[p], g[p]

In [10]:
def feature_extraction(sequence, tss):
    aacg = 0
    catt = 0
    gaac = 0
    gagc = 0
    tgcg = 0
    ttac = 0
    ttat = 0
    tttt = 0
    
    promoter_flag = 0
    bubble_flag = 0
    ar_overlap = 0
    ca_overlap = 0
    
    if (tss >= -35) and (tss <= -10):
        promoter_flag = 1
    elif (tss - 11 > -35) and (tss - 11) < -10:
        promoter_flag = 1
    elif (tss+11) > -35 and (tss+11) < -10:
        promoter_flag = 1
    
    if((tss >= -10) and (tss <= 2)):
        bubble_flag = 1
    elif(((tss-11) > -10) and ((tss-11) < 2)):
        bubble_flag = 1
    elif(((tss+11) > -10) and ((tss+11) < 2)):
        bubble_flag = 1
        
    if((tss >= -60) and (tss <= 60)):
        right = abs(tss+60)
        left = abs(60-tss)
        if(right>11):
            right = 11
        if(left>11):
            left = 11;

        ar_overlap = abs(right+left)
        
    elif(((tss-11) > -60) and ((tss-11) < 60)):
        ar_overlap = abs(60-(tss-11))
    elif(((tss+11) > -60) and ((tss+11) < 60)):
        ar_overlap = abs((tss+11)+60)

    if((tss >= -95) and (tss <= -35)):
        right = abs(tss+95)
        left = abs(-35-tss)
        if(right > 11):
            right = 11

        if(left > 11):
            left = 11

        ca_overlap = abs(left + right)
    elif(((tss-11) > -95) and ((tss-11) < -35)):
        ca_overlap = abs(-35-(tss-11))
    elif(((tss+11) > -95) and ((tss+11) < -35)):
        ca_overlap = abs((tss+11)+95)

    return aacg, catt, gaac, gagc, tgcg, ttac, ttat, tttt, promoter_flag, bubble_flag, ar_overlap, ca_overlap

In [11]:
promoters_is = []
for i, x in enumerate(CRP_TFB):
    if x[8] == "+":
        promoters_is.append((i, 0))
    else:
        promoters_is.append((i, 1))
        
unrelated_is = []
for i, _ in enumerate(unrelated_TFB):
    if x[8] == "+" or x[8] == "-":
        unrelated_is.append((i, 1))
    
seqs = []
befores = []
crps = []
afters = []
rules = []
locs = []
labels = []

for i in promoters_is:
    if len(CRP_TFB[i[0]]) > 10:
        if CRP_TFB[i[0]][10] != "":
            crp = CRP_TFB[i[0]][11]
            before, crp, after = get_split_dna(crp)
            promoter = split_by_upper(CRP_Promoters[i[0]][5]).upper() 
            loc = float(CRP_TFB[i[0]][10])
            locs.append([loc])
            seqs.append(dna_to_vectors(promoter))
            rules.append(feature_extraction(crp, loc))
            labels.append(i[1])
            befores.append(dna_to_vectors(before))
            crps.append(dna_to_vectors(crp))
            afters.append(dna_to_vectors(after))

random.shuffle(unrelated_is)
for i in unrelated_is:
    try:
        crp = unrelated_TFB[i[0]][11]
        before, crp, after = get_split_dna(crp)
        if len(crp) > 23:
            raise Exception
        loc = float(unrelated_TFB[i[0]][10])
        promoter = split_by_upper(unrelated_Promoters[i[0]][5]).upper() 
        locs.append([loc])
        seqs.append(dna_to_vectors(promoter))
        labels.append(i[1])
        rules.append(feature_extraction(crp, loc))
        befores.append(dna_to_vectors(before))
        crps.append(dna_to_vectors(crp))
        afters.append(dna_to_vectors(after))

    except Exception:
        pass

seqs = K.preprocessing.sequence.pad_sequences(seqs)
befores = K.preprocessing.sequence.pad_sequences(befores)
crps =  K.preprocessing.sequence.pad_sequences(crps)
afters =  K.preprocessing.sequence.pad_sequences(afters)
rules = np.asarray(rules)
labels = np.asarray(labels)
locs = np.asarray(locs)


befores, crps, afters, rules, labels, locs, seqs = unison_shuffled_copies(befores, crps, afters, rules, labels, locs, seqs)

C = 500

test_seqs = seqs[:C]
test_befores = befores[:C]
test_crps = crps[:C]
test_afters = afters[:C]
test_rules = rules[:C]
test_labels = labels[:C]
test_locs = locs[:C]

seqs = seqs[C:]
befores = befores[C:]
crps = crps[C:]
afters = afters[C:]
rules = rules[C:]
labels = labels[C:]
locs = locs[C:]



In [12]:
print(seqs.shape)
print(befores.shape)
print(crps.shape)
print(afters.shape)
print(rules.shape)
print(labels.shape)
print(locs.shape)

(2503, 60, 4)
(2503, 10, 4)
(2503, 23, 4)
(2503, 10, 4)
(2503, 12)
(2503,)
(2503, 1)


In [13]:
model.fit([seqs, locs, befores, crps, afters, rules], labels, validation_data = ([test_seqs,test_locs, test_befores, test_crps, test_afters, test_rules], test_labels), shuffle = True, epochs = 20, class_weight = "balanced")

Train on 2503 samples, validate on 500 samples
Epoch 1/20
2503/2503 [==============================] - 1s 550us/step - loss: 0.5401 - acc: 0.8174 - val_loss: 0.4086 - val_acc: 0.9260
Epoch 2/20
2503/2503 [==============================] - 1s 354us/step - loss: 0.3410 - acc: 0.9569 - val_loss: 0.3329 - val_acc: 0.9440
Epoch 3/20
2503/2503 [==============================] - 1s 371us/step - loss: 0.2858 - acc: 0.9596 - val_loss: 0.3295 - val_acc: 0.9400
Epoch 4/20
2503/2503 [==============================] - 1s 386us/step - loss: 0.2569 - acc: 0.9636 - val_loss: 0.2845 - val_acc: 0.9460
Epoch 5/20
2503/2503 [==============================] - 1s 368us/step - loss: 0.2280 - acc: 0.9692 - val_loss: 0.2689 - val_acc: 0.9460
Epoch 6/20
2503/2503 [==============================] - 1s 373us/step - loss: 0.2256 - acc: 0.9676 - val_loss: 0.2507 - val_acc: 0.9500
Epoch 7/20
2503/2503 [==============================] - 1s 376us/step - loss: 0.2031 - acc: 0.9720 - val_loss: 0.2528 - val_acc: 0.9500
E

In [14]:
predictions = model.predict([test_seqs,test_locs, test_befores, test_crps, test_afters, test_rules])
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(test_labels, np.round(predictions))
print(confusion)
tn = confusion[0][0]
fp = confusion[0][1]
tp = confusion[1][1]
fn = confusion[1][0]

sens = tn/(tn+fp)
spec = tp/(tp+fn)
ppv = tn/(tn+fn)
npv = tp/(tp+fp)

print(sens, spec, ppv, npv)


[[ 33   6]
 [ 13 448]]
0.8461538461538461 0.9718004338394793 0.717391304347826 0.986784140969163
